In [ ]:
data = read.csv("0914_categorized.csv")

library(dplyr)
library(xgboost)
library(DT)
library(Matrix)

data = data[,-1]

train = data[data$label != "testset", ] 
data3 = train  %>% filter(length == 3) %>% unique()
wide3 = data3  %>% group_by(acc_id) %>% mutate(index = row_number()) %>% as.data.frame()
wide3 = reshape(wide3, idvar = "acc_id", timevar = "index", direction = "wide")
id_3 = wide3 %>% dplyr::select(acc_id)
wide3 = wide3 %>% dplyr::select(-wk.1, -wk.2, -wk.3, -acc_id)
wide3$label.1 = as.integer(wide3$label.1) - 1
wide3$label.1[which(wide3$label.1 == 4)] = 3
ans3         = wide3$label.1
wide3         = wide3 %>% dplyr::select(-label.1, -label.2, -label.3)
data_matrix3  = xgb.DMatrix(data  = as.matrix(wide3), 
                             label = ans3)  

f1_calcul = function(table_eval){
  {
    fffmea = 8 / {
      1 / (table_eval[1,1] / sum(table_eval[,1])) +
        1 / (table_eval[2,2] / sum(table_eval[,2])) +
        1 / (table_eval[3,3] / sum(table_eval[,3])) +
        1 / (table_eval[4,4] / sum(table_eval[,4])) +
        1 / (table_eval[1,1] / sum(table_eval[1,])) +
        1 / (table_eval[2,2] / sum(table_eval[2,])) +
        1 / (table_eval[3,3] / sum(table_eval[3,])) +
        1 / (table_eval[4,4] / sum(table_eval[4,]))
    } 
    return(fffmea)
  }}

as.numeric.factor = function(x) {as.numeric(levels(x))[x]}

f1.max = function(preds, dtrain) {
  labels = getinfo(dtrain, "label")
  preds = matrix(preds, ncol=4, byrow=TRUE)
  table_eval = table(apply(preds,1,which.max)-1, labels)
  err = f1_calcul(table_eval)
  return(list(metric = "f1", value = err))
}

best_param = list()
best_seednumber = 1234
best_f1score = 0
best_f1score_index = 0
library(xgboost)
for (iter in 1:10) 
    
{
  xgb_params3 <- list(objective = "multi:softprob",
                eval_metric = f1.max,
                num_class = 4,
                max_depth = sample(40:60, 1),
                eta = runif(1, .01, .3),
                gamma = runif(1, 0.0, 0.2), 
                subsample = runif(1, .6, .9),
                colsample_bytree = runif(1, .5, .8), 
                min_child_weight = sample(1:40, 1),
                max_delta_step = sample(1:10, 1)
  )
  cv.nround = 1000
  cv.nfold = 5
  seed.number = sample.int(10000, 1)[[1]]
  set.seed(seed.number)
  mdcv <- xgb.cv(data=data_matrix3, label = ans3, params = xgb_params3, 
                 nfold=cv.nfold, nrounds=cv.nround, prediction = TRUE, 
                 maximize = TRUE, verbose = TRUE,
                 early_stopping_rounds=8,
                 print_every_n = 5,
                 callback(cb.cv.predict(save_models = TRUE)))
  
  max_f1score = max(mdcv$evaluation_log[, test_f1_mean])
  max_f1score_index = which.min(mdcv$evaluation_log[, test_f1_mean])
   
  if (max_f1score > best_f1score) {
   best_f1score = max_f1score
   best_f1score_index = max_f1score_index
   best_seednumber = seed.number
   best_param = xgb_params3
  }
}